In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import os
from urllib.parse import urljoin
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [3]:
def preprocess_for_training(data_file, output_file="training_data_processed.csv"):
    """
    Preprocess the scraped data for model training
    - Clean text
    - Expand query variations
    """
    print(f"Loading data from {data_file}...")
    
    # Load the data
    if data_file.endswith('.csv'):
        data = pd.read_csv(data_file).to_dict('records')
    elif data_file.endswith('.json'):
        data = pd.read_json(data_file).to_dict('records')
    else:
        raise ValueError("Data file must be CSV or JSON")
    
    processed_data = []
    
    print(f"Processing {len(data)} commands...")
    
    for cmd in data:
        # Basic cleaning
        description = re.sub(r'\s+', ' ', cmd['description']).strip()
        
        # Remove any HTML that might have been captured
        description = re.sub(r'<.*?>', '', description)
        
        # Create multiple variations of natural language queries
        variations = [
            f"How do I {description.lower().rstrip('.')}?",
            f"Show me how to {description.lower().rstrip('.')}",
            f"What command {description.lower().rstrip('.')}?",
            f"I need to {description.lower().rstrip('.')}"
        ]
        
        # Add command-specific variations
        if cmd['command'].lower() in ['dir', 'ls']:
            variations.extend([
                "List files in the current directory",
                "Show me what files are in this folder",
                "Display directory contents",
                "What's in this directory",
                "Show folder contents"
            ])
        
        if cmd['command'].lower() in ['cd', 'chdir']:
            variations.extend([
                "Change to a different directory",
                "Move to another folder",
                "Switch directories",
                "Navigate to directory",
                "Go to folder"
            ])
            
        if cmd['command'].lower() == 'copy':
            variations.extend([
                "Copy a file from one location to another",
                "Make a copy of a file",
                "Duplicate a file",
                "Copy files"
            ])
            
        if cmd['command'].lower() in ['del', 'erase']:
            variations.extend([
                "Delete a file",
                "Remove a file",
                "Get rid of a file",
                "Erase files"
            ])
            
        if cmd['command'].lower() == 'ipconfig':
            variations.extend([
                "Show my IP address",
                "What's my network configuration",
                "Display network settings",
                "Check IP configuration"
            ])
        
        # Add each variation as a training example
        for query in variations:
            processed_data.append({
                'query': query,
                'command': cmd['command'],
                'description': description,
                'syntax': cmd.get('syntax', '')
            })
    
    # Save to CSV
    df = pd.DataFrame(processed_data)
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Processed {len(processed_data)} training examples saved to {output_file}")
    
    return df

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

In [5]:
def train_command_model(training_data, model_dir="model"):
    """
    Train a model to predict commands from natural language queries
    
    Args:
        training_data: DataFrame or path to CSV file with training data
        model_dir: Directory to save the model files
    """
    print("Training model...")
    
    # Load the data if it's a file path
    if isinstance(training_data, str):
        if training_data.endswith('.csv'):
            df = pd.read_csv(training_data)
        else:
            raise ValueError("Training data file must be CSV")
    else:
        df = training_data
    
    print(f"Using {len(df)} training examples")
    
    # Split features and target
    X = df['query']
    y = df['command']
    
    # Split into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    print(f"Training set size: {len(X_train)}, Test set size: {len(X_test)}")
    
    # Convert text to features
    vectorizer = TfidfVectorizer(ngram_range=(1, 3))
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Train a classifier
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train_vec, y_train)
    
    # Evaluate the model
    y_pred = clf.predict(X_test_vec)
    print("\nModel performance:")
    print(classification_report(y_test, y_pred))
    
    # Create model directory if it doesn't exist
    os.makedirs(model_dir, exist_ok=True)
    
    # Save the model and vectorizer
    model_file = os.path.join(model_dir, "command_model.pkl")
    vectorizer_file = os.path.join(model_dir, "command_vectorizer.pkl")
    
    joblib.dump(clf, model_file)
    joblib.dump(vectorizer, vectorizer_file)
    
    print(f"Model saved to {model_file}")
    print(f"Vectorizer saved to {vectorizer_file}")
    
    return clf, vectorizer

In [6]:

def predict_command(query, clf=None, vectorizer=None):
    """Predict the appropriate command for a given query"""
    # Load the model if not provided
    if clf is None or vectorizer is None:
        clf = joblib.load('data/command_model.pkl')
        vectorizer = joblib.load('data/command_vectorizer.pkl')
    
    # Process the query
    query_vec = vectorizer.transform([query])
    
    # Make the prediction
    command = clf.predict(query_vec)[0]
    
    # Get confidence scores
    proba = clf.predict_proba(query_vec)[0]
    confidence = max(proba)
    
    return command, confidence

In [7]:
def create_cli(command_data_file=None, model_dir="model"):
    """
    Create an interactive CLI to test the model
    
    Args:
        command_data_file: File with detailed command information (optional)
        model_dir: Directory where model files are stored
    """
    print("\nNatural Language to Windows Command System")
    print("Type 'exit' to quit\n")
    
    # Load detailed command info if available
    command_dict = {}
    if command_data_file:
        if command_data_file.endswith('.csv'):
            df = pd.read_csv(command_data_file)
        elif command_data_file.endswith('.json'):
            df = pd.read_json(command_data_file)
        else:
            print("Warning: Unsupported file format. Command details won't be available.")
            df = None
            
        if df is not None:
            command_dict = {row['command']: row for _, row in df.iterrows()}
    
    while True:
        user_input = input("What would you like to do? ")
        
        if user_input.lower() in ['exit', 'quit', 'q']:
            print("Goodbye!")
            break
        
        # Predict the command
        try:
            command, confidence = predict_command(user_input, model_dir)
            
            # Display information
            print(f"\nSuggested command: {command}")
            print(f"Confidence: {confidence:.2f}")
            
            if command in command_dict:
                cmd_info = command_dict[command]
                print(f"\nDescription: {cmd_info['description']}")
                
                if 'syntax' in cmd_info and cmd_info['syntax']:
                    print(f"\nSyntax:")
                    print(cmd_info['syntax'])
                
                if 'examples' in cmd_info and isinstance(cmd_info['examples'], list) and len(cmd_info['examples']) > 0:
                    print("\nExample:")
                    print(f"  {cmd_info['examples'][0]}")
            
        except Exception as e:
            print(f"Error: {str(e)}")
        
        print("\n" + "-"*50)

In [ ]:
def main():
    """
    Main function to run the preprocessing and model training pipeline
    """
    # Get input file from user
    print("Windows Command NLP Model Training")
    print("=================================")
    
    data_file = input("Enter path to your scraped data file (CSV or JSON): ")
    
    # Check if the file exists
    if not os.path.exists(data_file):
        print(f"Error: File '{data_file}' not found!")
        return
    
    # Preprocess data
    processed_data = preprocess_for_training(data_file)
    
    # Train the model
    clf, vectorizer = train_command_model(processed_data)
    
    # Start the CLI for testing
    create_cli(data_file)

if __name__ == "__main__":
    main()

Windows Command NLP Model Training
Loading data from data/windows_commands_detailed.json...
Processing 258 commands...
Processed 1063 training examples saved to training_data_processed.csv
Training model...
Using 1063 training examples
Training set size: 850, Test set size: 213


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMet


Model performance:
                   precision    recall  f1-score   support

         ADmodcmd       1.00      1.00      1.00         1
              ARP       1.00      1.00      1.00         1
           ATTRIB       1.00      1.00      1.00         1
         AUDITPOL       1.00      1.00      1.00         1
        AccessChk       1.00      1.00      1.00         2
          BCDBOOT       1.00      1.00      1.00         2
        BITSADMIN       1.00      1.00      1.00         1
            CACLS       1.00      1.00      1.00         1
             CALL       1.00      1.00      1.00         2
               CD       0.00      0.00      0.00         3
          CERTREQ       1.00      1.00      1.00         2
         CERTUTIL       1.00      1.00      1.00         1
           CHANGE       1.00      1.00      1.00         1
         CHANGEPK       1.00      1.00      1.00         1
             CHCP       1.00      1.00      1.00         1
            CHDIR       0.20      0

In [ ]:


query = "how do I check my IP address"
command, confidence = predict_command(query)
print(f"Command: {command}, Confidence: {confidence:.2f}")

ModuleNotFoundError: No module named 'windows_cmd_model'